### Libraries

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Hyperparameter

In [ ]:
batch_size = 25
reduction_dim = 30
lr = 0.002

### Get Data

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5], [0.5]
        )
    ]
)

folder_path = "data"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
loader = torch.utils.data.DataLoader(dataset=dataset, batch_size = 32, shuffle = True)

### Autoencoder

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 9)
        )
        
        self.decoder = nn.Sequential(
            torch.nn.Linear(9, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28 * 28),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

### Network

In [ ]:
data_dim = 1
# automatically calculate the dimension
for dimension in range(1, dataset_train.data.ndim):
    data_dim *= dataset_train.data.size(dimension)

AE = AutoEncoder()
lossFunction = nn.BCELoss()

# Optimizers

Gen_optimizer = optim.Adam(E.parameters(), lr=lr)
Dis_optimizer = optim.Adam(D.parameters(), lr=lr)

### Train